In [192]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.optimize as opt
from sklearn.model_selection import train_test_split
import random as rnd

In [193]:
data = pd.read_excel("C:/Users/Niku/Documents/dataset/arrays.xlsx")
data.shape

(1992, 36)

In [194]:
def kernel_linear(x1, x2):
    return np.dot(x1, x2.T)
def kernel_quadratic(x1, x2):
    return (np.dot(x1, x2.T) ** 2)

In [195]:
def rnd_int(a,b,z):
    i = z
    while i == z:
        i = rnd.randint(a,b)
    return i

In [196]:
def predict(X):
    return h(X, w, b)

In [197]:
def calc_b(X, y, w):
    b_tmp = y - np.dot(w.T, X.T)
    return np.mean(b_tmp)

In [198]:
def calc_w(alpha, y, X):
    return np.dot(alpha * y, X)

In [199]:
def h(X, w, b):
    return np.sign(np.dot(w.T, X.T) + b).astype(int)

In [200]:
def E(x_k, y_k, w, b):
    return h(x_k, w, b) - y_k

In [201]:
 def compute_L_H(C, alpha_prime_j, alpha_prime_i, y_j, y_i):
    if(y_i != y_j):
        return (max(0, alpha_prime_j - alpha_prime_i), min(C, C - alpha_prime_i + alpha_prime_j))
    else:
        return (max(0, alpha_prime_i + alpha_prime_j - C), min(C, alpha_prime_i + alpha_prime_j))

In [202]:
max_iter=10000
kernel_type='linear'
C=1.0
epsilon=0.001

In [203]:
def fit(X, y):
    # Initialization
    n, d = X.shape[0], X.shape[1]
    alpha = np.zeros((n))
    #alpha = np.array(alpha)
    #print (alpha.shape)
    count = 0
    while True:
        count += 1
        alpha_prev = np.copy(alpha)
        for j in range(0, n):
            i = rnd_int(0, n-1, j) # Get random int i~=j
            x_i, x_j, y_i, y_j = X[i,:], X[j,:], y[i], y[j]
            #k_ij = kernel_quadratic(x_i, x_i) + kernel_quadratic(x_j, x_j) - 2 * kernel_quadratic(x_i, x_j)
            k_ij = kernel_linear(x_i, x_i) + kernel_linear(x_j, x_j) - 2 * kernel_linear(x_i, x_j)
            if k_ij == 0:
                continue
            alpha_prime_j, alpha_prime_i = alpha[j], alpha[i]
            (L, H) = compute_L_H(C, alpha_prime_j, alpha_prime_i, y_j, y_i)

                # Compute model parameters
            w = calc_w(alpha, y, X)
            b = calc_b(X, y, w)
            
                # Compute E_i, E_j
            E_i = E(x_i, y_i, w, b)
            E_j = E(x_j, y_j, w, b)

                # Set new alpha values
            alpha[j] = alpha_prime_j + float(y_j * (E_i - E_j))/k_ij
            alpha[j] = max(alpha[j], L)
            alpha[j] = min(alpha[j], H)

            alpha[i] = alpha_prime_i + y_i*y_j * (alpha_prime_j - alpha[j])

            # Check convergence
        diff = np.linalg.norm(alpha - alpha_prev)
        if diff < epsilon:
            break

        if count >= max_iter:
            print("Iteration number exceeded the max of %d iterations" % (max_iter))
            return
        # Compute final model parameters
    b = calc_b(X, y, w)
    if kernel_type == 'linear':
        w = calc_w(alpha, y, X)
    # Get support vectors
    alpha_idx = np.where(alpha > 0)[0]
    support_vectors = X[alpha_idx, :]
    return b, w

In [204]:
X = data[0:1992]
X = np.array(X)
y = X[:, 35]
X = X[:, 0:2]
X.shape
#y = np.matrix(y)
y = np.array(y)
y[y == 0] = -1
y1 = np.matrix(y)
y.shape, X.shape, y1.shape

((1992L,), (1992L, 2L), (1L, 1992L))

In [205]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.shape, y_train.shape

((1593L, 2L), (1593L,))

In [206]:
[b, w] = fit(X_train, y_train)

In [207]:
y_pred = predict(X_test)

In [208]:
u1, c1 = np.unique(y_pred, return_counts=True)
print u1, c1
u2, c2 = np.unique(np.array(y_test), return_counts=True)
print u2, c2

[-1  1] [357  42]
[-1.  1.] [320  79]


In [209]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y_test)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print 'accuracy = {0}%'.format(accuracy * 100)

accuracy = 85.2130325815%


In [210]:
y_pred[0:10]

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [211]:
y_test[0:10]

array([-1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.])

In [190]:
np.sum(correct) 

323

In [191]:
len(correct)

399